In [13]:
import sys
sys.api_version
!{sys.executable} -m pip install -qUr requirements.txt

In [14]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain_openai import OpenAIEmbeddings
from package import OPENAI_API_KEY, set_environment_openai_api_key
from package.utilities import get_file_contents
set_environment_openai_api_key()

chat = ChatOpenAI(model='gpt-4o', openai_api_key=OPENAI_API_KEY)
orchestrator_prompt = get_file_contents('orchestrator-prompt.txt')
entities = get_file_contents('entities.txt').splitlines()

In [15]:
aiMessage: AIMessage = chat.invoke([
    SystemMessage(content=orchestrator_prompt),
    HumanMessage(content="The manager of chat-bot project has been changed, now it's Ivan Petrov")
])
candidates = [candidate.lstrip('0123456789.- ') for candidate in aiMessage.content.splitlines()]
print(candidates)

['manager', 'chat-bot project', 'Ivan Petrov']


In [16]:
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", headers={"x-api-key": OPENAI_API_KEY})
retriever = FAISS.from_texts(entities, embeddings).as_retriever()
for candidate in candidates:
    print(candidate, ' -> ', retriever.invoke(candidate)[0].page_content)

manager  ->  director
chat-bot project  ->  bot program
Ivan Petrov  ->  person name
